# Project 11 (Ultimate) — Phoenix Autocall under Smile  
### SVI implied vol → Dupire local vol → Monte Carlo (with a visual dashboard)

This notebook is designed to be **GitHub-ready** and **highly visual**:

- ✅ Phoenix Autocall payoff (coupon barrier + memory + autocall + protection barrier)  
- ✅ **Smile** via a smooth **SVI** implied-vol surface (synthetic by default → offline friendly)  
- ✅ **Local volatility** via a pragmatic **Dupire** construction (finite differences on call prices)  
- ✅ Monte Carlo pricing under:
  - **GBM (const vol)** baseline
  - **Local Vol** (smile-consistent)
- ✅ Visuals:
  - 3D **vol “sheet”** (IV + LocalVol)
  - **Animated smile** across maturities (Play button)
  - Path fan charts + barrier overlays
  - Call probability by date
  - PV / model-diff distributions
  - Sensitivity heatmap (coupon × autocall barrier)
  - Convergence chart

> Tip (VS Code): install extensions **Python** + **Jupyter**, and run this notebook with a Python kernel that has:  
`numpy pandas plotly ipywidgets`


In [1]:
# 0) Setup: imports + options
from __future__ import annotations

import math
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Tuple, Optional

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False
    widgets = None
    display = None
    Markdown = None

SEED = 42
rng = np.random.default_rng(SEED)

np.set_printoptions(suppress=True, precision=6)

ASSETS_DIR = Path.cwd() / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("ASSETS_DIR:", ASSETS_DIR)
print("ipywidgets available:", HAS_WIDGETS)


ASSETS_DIR: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\11_Phoenix_Autocall\assets
ipywidgets available: True


---  
## 1) Product definition — Phoenix Autocall

### 1.1 Payoff rules (typical Phoenix)
Observation dates: \(0 < t_1 < \cdots < t_m = T\).

- **Coupon barrier** \(B_{\text{cpn}}\): if \(S_{t_k} \ge B_{\text{cpn}} S_0\), pay coupon \(cN\).
- **Memory (optional)**: missed coupons accumulate; when barrier is met later, pay \((\text{missed}+1)cN\).
- **Autocall barrier** \(B_{\text{call}}\): if \(S_{t_k} \ge B_{\text{call}} S_0\), redeem early: pay notional \(N\) (and typically the coupon).
- **Protection barrier** \(B_{\text{prot}}\) at maturity:
  \[
  \text{Redemption} =
  \begin{cases}
  N, & S_T \ge B_{\text{prot}} S_0,\\
  N\cdot S_T/S_0, & S_T < B_{\text{prot}} S_0.
  \end{cases}
  \]

We price in a risk-neutral setting (discount at \(r\)).

### 1.2 Quick payoff intuition plot (maturity redemption only)
This plot shows the **capital mechanism** at maturity (if not called).


In [2]:
@dataclass
class PhoenixAutocallSpec:
    notional: float = 100.0
    maturity: float = 1.0
    obs_per_year: int = 12
    coupon: float = 0.012
    B_coupon: float = 0.70
    B_call: float = 1.00
    B_prot: float = 0.60
    memory: bool = True
    include_coupon_at_call: bool = True


def plot_maturity_redemption(S0: float, spec: PhoenixAutocallSpec):
    x = np.linspace(0.2*S0, 1.6*S0, 400)
    y = np.where(x >= spec.B_prot*S0, spec.notional, spec.notional*(x/S0))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="Redemption at T"))
    fig.add_hline(y=spec.notional, line_dash="dash", annotation_text="Notional", opacity=0.5)
    fig.add_vline(x=spec.B_prot*S0, line_dash="dash", annotation_text="Protection barrier", opacity=0.7)
    fig.update_layout(
        template="plotly_dark",
        title="Maturity redemption profile (if not called)",
        xaxis_title="Terminal spot S_T",
        yaxis_title="Redemption (currency units)"
    )
    return fig


S0_demo = 100.0
spec_demo = PhoenixAutocallSpec()
fig = plot_maturity_redemption(S0_demo, spec_demo)
fig.show()
fig.write_html(ASSETS_DIR / "p11_ultimate_maturity_redemption.html")


c:\Users\Karim\anaconda3\Lib\site-packages\kaleido\_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




---  
## 2) Models — GBM baseline vs Smile-aware Local Vol

### 2.1 Risk-neutral GBM (baseline)
\[
dS_t = r S_t\,dt + \sigma S_t\,dW_t
\]
Exact step:
\[
S_{t+\Delta t} = S_t \exp\Big((r-\tfrac12\sigma^2)\Delta t + \sigma\sqrt{\Delta t}\,Z\Big),\quad Z\sim\mathcal{N}(0,1).
\]

### 2.2 Smile and implied volatility surface
Market smiles imply \(\sigma_{\text{imp}}(T,K)\) depends on strike and maturity.

We use a smooth synthetic **SVI** surface (offline).
SVI models total variance \(w(T,k)=\sigma_{\text{imp}}^2 T\) as:
\[
w(T,k)=a+b\big(\rho(k-m)+\sqrt{(k-m)^2+\eta^2}\big), \qquad k=\ln(K/F_T).
\]

### 2.3 Dupire local volatility (smile-consistent dynamics)
We build \(\sigma_{\text{loc}}(T,K)\) from call prices \(C(T,K)\) via:
\[
\sigma_{\text{loc}}^2(T,K)=\frac{\partial_T C + rK\partial_K C}{\tfrac12 K^2 \partial_{KK}C}.
\]
Then simulate:
\[
dS_t = r S_t\,dt + \sigma_{\text{loc}}(t,S_t) S_t\, dW_t.
\]

> Important: Dupire is numerically sensitive → we smooth and clip for stability.


In [3]:
# 2) Core math utilities (no SciPy required)

def _norm_cdf(x: np.ndarray) -> np.ndarray:
    return 0.5 * (1.0 + np.vectorize(math.erf)(x / math.sqrt(2.0)))


def bs_call_price(S0: float, K: np.ndarray, T: float, r: float, sigma: np.ndarray) -> np.ndarray:
    K = np.asarray(K, dtype=float)
    sigma = np.asarray(sigma, dtype=float)
    sigma = np.maximum(sigma, 1e-12)
    T = float(max(T, 1e-12))
    sqrtT = math.sqrt(T)
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return S0 * _norm_cdf(d1) - K * np.exp(-r * T) * _norm_cdf(d2)


def svi_total_variance(k: np.ndarray, a: float, b: float, rho: float, m: float, eta: float) -> np.ndarray:
    return a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + eta**2))


def build_synthetic_svi_surface(
    S0: float,
    r: float,
    T_list: np.ndarray,
    K_list: np.ndarray,
    vol_level: float = 0.22,
    skew: float = -0.45,
    curvature: float = 0.70,
) -> np.ndarray:
    sig_imp = np.zeros((len(T_list), len(K_list)), dtype=float)
    for i, T in enumerate(T_list):
        F = S0 * np.exp(r * T)
        k = np.log(K_list / F)

        rho = float(np.clip(skew, -0.999, 0.999))
        b = 0.10 + 0.22 * curvature * (0.25 + T)
        m = -0.05 * rho
        eta = 0.25 + 0.25 * curvature
        a = max((vol_level**2) * T - b * eta, 1e-6)

        w = svi_total_variance(k, a=a, b=b, rho=rho, m=m, eta=eta)
        w = np.maximum(w, 1e-8)
        sig_imp[i, :] = np.sqrt(w / T)
    return sig_imp


In [4]:
# 2.4) Finite differences + smoothing for Dupire

def finite_diff_first(x: np.ndarray, y: np.ndarray, axis: int) -> np.ndarray:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    dy = np.zeros_like(y)

    if axis == 0:
        for i in range(y.shape[0]):
            if i == 0:
                dy[i, :] = (y[1, :] - y[0, :]) / (x[1] - x[0])
            elif i == y.shape[0] - 1:
                dy[i, :] = (y[-1, :] - y[-2, :]) / (x[-1] - x[-2])
            else:
                dy[i, :] = (y[i + 1, :] - y[i - 1, :]) / (x[i + 1] - x[i - 1])
    else:
        for j in range(y.shape[1]):
            if j == 0:
                dy[:, j] = (y[:, 1] - y[:, 0]) / (x[1] - x[0])
            elif j == y.shape[1] - 1:
                dy[:, j] = (y[:, -1] - y[:, -2]) / (x[-1] - x[-2])
            else:
                dy[:, j] = (y[:, j + 1] - y[:, j - 1]) / (x[j + 1] - x[j - 1])
    return dy


def finite_diff_second(x: np.ndarray, y: np.ndarray, axis: int) -> np.ndarray:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    d2 = np.zeros_like(y)

    if axis == 1:
        for j in range(1, y.shape[1] - 1):
            dx1 = x[j] - x[j - 1]
            dx2 = x[j + 1] - x[j]
            d2[:, j] = 2 * ((y[:, j + 1] - y[:, j]) / dx2 - (y[:, j] - y[:, j - 1]) / dx1) / (dx1 + dx2)
        d2[:, 0] = d2[:, 1]
        d2[:, -1] = d2[:, -2]
    else:
        for i in range(1, y.shape[0] - 1):
            dx1 = x[i] - x[i - 1]
            dx2 = x[i + 1] - x[i]
            d2[i, :] = 2 * ((y[i + 1, :] - y[i, :]) / dx2 - (y[i, :] - y[i - 1, :]) / dx1) / (dx1 + dx2)
        d2[0, :] = d2[1, :]
        d2[-1, :] = d2[-2, :]
    return d2


def smooth_2d(A: np.ndarray, passes: int = 2) -> np.ndarray:
    B = A.copy()
    for _ in range(passes):
        C = B.copy()
        C[1:-1, 1:-1] = (
            B[1:-1, 1:-1]
            + B[:-2, 1:-1]
            + B[2:, 1:-1]
            + B[1:-1, :-2]
            + B[1:-1, 2:]
        ) / 5.0
        B = C
    return B


def build_local_vol_from_iv(
    S0: float,
    r: float,
    T_grid: np.ndarray,
    K_grid: np.ndarray,
    sigma_imp: np.ndarray,
    clip_min: float = 0.03,
    clip_max: float = 1.50,
) -> np.ndarray:
    # Call price grid
    C = np.zeros_like(sigma_imp)
    for i, T in enumerate(T_grid):
        C[i, :] = bs_call_price(S0, K_grid, float(T), r, sigma_imp[i, :])

    dC_dT = finite_diff_first(T_grid, C, axis=0)
    dC_dK = finite_diff_first(K_grid, C, axis=1)
    d2C_dK2 = finite_diff_second(K_grid, C, axis=1)

    K = K_grid[None, :]
    denom = 0.5 * (K**2) * d2C_dK2
    numer = dC_dT + r * K * dC_dK

    eps = 1e-10
    sigma2 = np.where(np.abs(denom) > eps, numer / denom, np.nan)

    finite = np.isfinite(sigma2)
    fill = np.nanmedian(sigma2[finite]) if finite.any() else 0.04
    sigma2 = np.nan_to_num(sigma2, nan=fill)

    sigma2 = np.clip(sigma2, clip_min**2, clip_max**2)
    lv = np.sqrt(sigma2)

    lv = smooth_2d(lv, passes=2)
    return np.clip(lv, clip_min, clip_max)


class LocalVolInterpolator:
    """Bilinear interpolation on (T, K) grid. We query with spot as K proxy."""
    def __init__(self, T_grid: np.ndarray, K_grid: np.ndarray, sigma_loc: np.ndarray):
        self.T = np.asarray(T_grid, float)
        self.K = np.asarray(K_grid, float)
        self.S = np.asarray(sigma_loc, float)  # shape (T,K)

    def __call__(self, t: float, spot: np.ndarray) -> np.ndarray:
        t = float(np.clip(t, self.T[0], self.T[-1]))
        spot = np.asarray(spot, float)
        s = np.clip(spot, self.K[0], self.K[-1])

        i = int(np.clip(np.searchsorted(self.T, t) - 1, 0, len(self.T) - 2))
        t0, t1 = self.T[i], self.T[i + 1]
        wt = 0.0 if t1 == t0 else (t - t0) / (t1 - t0)

        j = np.clip(np.searchsorted(self.K, s) - 1, 0, len(self.K) - 2).astype(int)
        k0, k1 = self.K[j], self.K[j + 1]
        wk = np.where(k1 == k0, 0.0, (s - k0) / (k1 - k0))

        s00 = self.S[i, j]
        s01 = self.S[i, j + 1]
        s10 = self.S[i + 1, j]
        s11 = self.S[i + 1, j + 1]

        s0 = (1 - wk) * s00 + wk * s01
        s1 = (1 - wk) * s10 + wk * s11
        return (1 - wt) * s0 + wt * s1


---  
## 3) Ultimate visuals — the “vol sheet” + animated smile (Play)

This is the part you show on GitHub / in interviews:
- a 3D **implied vol sheet** \(\sigma_{\text{imp}}(T,K)\)
- a 3D **local vol sheet** \(\sigma_{\text{loc}}(T,K)\)
- an **animation** of the smile across maturities (Play button)

> These charts are interactive (rotate, zoom, hover).


In [5]:
def plot_vol_surface_3d(T_grid: np.ndarray, K_grid: np.ndarray, Z: np.ndarray, title: str, zlabel: str):
    fig = go.Figure(data=[go.Surface(
        x=K_grid,
        y=T_grid,
        z=Z,
        colorbar=dict(title=zlabel),
        showscale=True
    )])
    fig.update_layout(
        template="plotly_dark",
        title=title,
        scene=dict(
            xaxis_title="Strike / Spot level (K or S)",
            yaxis_title="Maturity T (years)",
            zaxis_title=zlabel
        ),
        height=650
    )
    return fig


def plot_smile_animation(T_grid: np.ndarray, K_grid: np.ndarray, sigma_imp: np.ndarray, title: str):
    # Frames: each frame is a smile slice at fixed T
    frames = []
    for i, T in enumerate(T_grid):
        frames.append(go.Frame(
            data=[go.Scatter(x=K_grid, y=sigma_imp[i, :], mode="lines+markers")],
            name=f"T={T:.3f}"
        ))

    fig = go.Figure(
        data=[go.Scatter(x=K_grid, y=sigma_imp[0, :], mode="lines+markers")],
        frames=frames
    )

    steps = []
    for fr in frames:
        steps.append(dict(
            method="animate",
            args=[[fr.name], dict(mode="immediate", frame=dict(duration=350, redraw=True), transition=dict(duration=0))],
            label=fr.name
        ))

    fig.update_layout(
        template="plotly_dark",
        title=title,
        xaxis_title="Strike K",
        yaxis_title="Implied vol σ_imp",
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            x=0.0, y=1.12,
            buttons=[
                dict(label="Play", method="animate",
                     args=[None, dict(frame=dict(duration=350, redraw=True), fromcurrent=True)]),
                dict(label="Pause", method="animate",
                     args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")]),
            ],
        )],
        sliders=[dict(steps=steps, x=0.05, y=-0.08, len=0.9)]
    )
    return fig


In [6]:
# Build a default surface (offline)
S0 = 100.0
r = 0.03
spec = PhoenixAutocallSpec(maturity=1.0, obs_per_year=12)

T_grid = np.linspace(1/12, spec.maturity, 12)
K_grid = np.linspace(0.5*S0, 1.5*S0, 81)

vol_level, skew, curvature = 0.22, -0.45, 0.70
sigma_imp = build_synthetic_svi_surface(S0, r, T_grid, K_grid, vol_level, skew, curvature)

sigma_loc = build_local_vol_from_iv(S0, r, T_grid, K_grid, sigma_imp)
lv = LocalVolInterpolator(T_grid, K_grid, sigma_loc)

fig_iv3d = plot_vol_surface_3d(T_grid, K_grid, sigma_imp, "Implied Vol Surface (SVI synthetic)", "σ_imp")
fig_iv3d.show()
fig_iv3d.write_html(ASSETS_DIR / "p11_ultimate_iv_surface_3d.html")

fig_lv3d = plot_vol_surface_3d(T_grid, K_grid, sigma_loc, "Local Vol Surface (Dupire)", "σ_loc")
fig_lv3d.show()
fig_lv3d.write_html(ASSETS_DIR / "p11_ultimate_localvol_surface_3d.html")

fig_anim = plot_smile_animation(T_grid, K_grid, sigma_imp, "Smile animation across maturities (Play)")
fig_anim.show()
fig_anim.write_html(ASSETS_DIR / "p11_ultimate_smile_animation.html")


---  
## 4) Monte Carlo engines (fast, vectorized payoff)

This is the key improvement:
- **common random numbers** between models (reduces noise in the model-diff)
- **vectorized payoff evaluation** across paths (much faster)
- optional **antithetic variates** (variance reduction)


In [7]:
def simulate_gbm_with_Z(S0: float, r: float, sigma: float, T: float, steps: int, Z: np.ndarray):
    # Z shape: (steps, n_paths)
    dt = T / steps
    t = np.linspace(0.0, T, steps + 1)
    S = np.empty((steps + 1, Z.shape[1]), float)
    S[0, :] = S0
    drift = (r - 0.5 * sigma**2) * dt
    vol = sigma * math.sqrt(dt)
    for k in range(steps):
        S[k + 1, :] = S[k, :] * np.exp(drift + vol * Z[k, :])
    return t, S


def simulate_localvol_with_Z(S0: float, r: float, lv: LocalVolInterpolator, T: float, steps: int, Z: np.ndarray):
    dt = T / steps
    t = np.linspace(0.0, T, steps + 1)
    S = np.empty((steps + 1, Z.shape[1]), float)
    S[0, :] = S0
    for k in range(steps):
        sig = np.maximum(lv(t[k], S[k, :]), 1e-6)
        drift = (r - 0.5 * sig**2) * dt
        vol = sig * math.sqrt(dt)
        S[k + 1, :] = S[k, :] * np.exp(drift + vol * Z[k, :])
    return t, S


def phoenix_price_vectorized(t: np.ndarray, S: np.ndarray, S0: float, r: float, spec: PhoenixAutocallSpec):
    n_paths = S.shape[1]
    m = int(spec.obs_per_year * spec.maturity)
    obs_times = np.linspace(spec.maturity / m, spec.maturity, m)
    obs_idx = np.searchsorted(t, obs_times, side="left")
    obs_idx = np.clip(obs_idx, 0, len(t) - 1)

    S_obs = S[obs_idx, :]  # (m, n_paths)

    pv = np.zeros(n_paths, float)
    alive = np.ones(n_paths, bool)
    called = np.zeros(n_paths, float)
    call_time = np.full(n_paths, spec.maturity, float)
    missed = np.zeros(n_paths, int)

    for k in range(m):
        tk = float(obs_times[k])
        St = S_obs[k, :]

        # Coupon
        trig_cpn = alive & (St >= spec.B_coupon * S0)
        if spec.memory:
            coupon_amt = (missed + 1) * spec.coupon * spec.notional
            coupon_paid = np.where(trig_cpn, coupon_amt, 0.0)
            missed = np.where(trig_cpn, 0, missed + alive.astype(int) * (~trig_cpn).astype(int))
        else:
            coupon_paid = np.where(trig_cpn, spec.coupon * spec.notional, 0.0)

        pv += np.exp(-r * tk) * coupon_paid

        # Autocall
        trig_call = alive & (St >= spec.B_call * S0)
        if trig_call.any():
            pv += np.exp(-r * tk) * spec.notional * trig_call.astype(float)
            called = np.where(trig_call, 1.0, called)
            call_time = np.where(trig_call, tk, call_time)
            alive = alive & (~trig_call)

    # Maturity redemption (only for not-called paths)
    if alive.any():
        ST = S[-1, :]
        redemption = np.where(ST >= spec.B_prot * S0, spec.notional, spec.notional * (ST / S0))
        pv += np.exp(-r * spec.maturity) * redemption * alive.astype(float)

    diag = pd.DataFrame({"pv": pv, "called": called, "call_time": call_time})
    return float(pv.mean()), diag


---  
## 5) Pricing + core diagnostics (GBM vs Local Vol)

We compute:
- prices (GBM, LocalVol)
- call probability curve
- PV distribution
- **model-diff distribution** (LocalVol PV − GBM PV) using common random numbers  
  → this is a very clean “smile risk” graphic.


In [8]:
def call_prob_by_date(diag: pd.DataFrame, spec: PhoenixAutocallSpec) -> pd.DataFrame:
    m = int(spec.obs_per_year * spec.maturity)
    obs_times = np.linspace(spec.maturity / m, spec.maturity, m)
    ct = diag["call_time"].to_numpy()
    called = diag["called"].to_numpy()
    rows = []
    for t_ in obs_times:
        rows.append({
            "t": t_,
            "p_call_at_t": float(((called > 0.5) & (np.abs(ct - t_) < 1e-9)).mean()),
        })
    out = pd.DataFrame(rows)
    out["p_call_by_t"] = out["p_call_at_t"].cumsum()
    return out


def fan_chart(t: np.ndarray, S: np.ndarray, title: str, S0: float, spec: PhoenixAutocallSpec):
    # Percentile bands
    q = np.quantile(S, [0.05, 0.25, 0.5, 0.75, 0.95], axis=1)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t, y=q[2], mode="lines", name="Median"))
    fig.add_trace(go.Scatter(x=t, y=q[4], mode="lines", name="95%", line=dict(width=1), opacity=0.6))
    fig.add_trace(go.Scatter(x=t, y=q[0], mode="lines", name="5%", line=dict(width=1), opacity=0.6, fill="tonexty"))
    fig.add_trace(go.Scatter(x=t, y=q[3], mode="lines", name="75%", line=dict(width=1), opacity=0.6))
    fig.add_trace(go.Scatter(x=t, y=q[1], mode="lines", name="25%", line=dict(width=1), opacity=0.6, fill="tonexty"))

    # Barriers
    fig.add_hline(y=spec.B_coupon*S0, line_dash="dot", opacity=0.7, annotation_text="Coupon barrier")
    fig.add_hline(y=spec.B_call*S0, line_dash="dash", opacity=0.7, annotation_text="Autocall barrier")
    fig.add_hline(y=spec.B_prot*S0, line_dash="dashdot", opacity=0.7, annotation_text="Protection barrier")

    fig.update_layout(template="plotly_dark", title=title, xaxis_title="t (years)", yaxis_title="S_t")
    return fig


def run_pricing_bundle(
    S0: float,
    r: float,
    spec: PhoenixAutocallSpec,
    steps: int,
    n_paths: int,
    vol_level: float,
    skew: float,
    curvature: float,
    antithetic: bool = True,
):
    # grids
    T_grid = np.linspace(1/spec.obs_per_year, spec.maturity, max(12, int(spec.obs_per_year*spec.maturity)))
    K_grid = np.linspace(0.5*S0, 1.5*S0, 81)

    sigma_imp = build_synthetic_svi_surface(S0, r, T_grid, K_grid, vol_level, skew, curvature)
    sigma_loc = build_local_vol_from_iv(S0, r, T_grid, K_grid, sigma_imp)
    lv = LocalVolInterpolator(T_grid, K_grid, sigma_loc)

    atm_idx = int(np.argmin(np.abs(K_grid - S0)))
    sigma_const = float(np.median(sigma_imp[:, atm_idx]))

    # common random numbers
    Z = np.random.default_rng(SEED).standard_normal((steps, n_paths))
    if antithetic:
        Z = np.concatenate([Z, -Z], axis=1)

    t_gbm, S_gbm = simulate_gbm_with_Z(S0, r, sigma_const, spec.maturity, steps, Z)
    t_lv,  S_lv  = simulate_localvol_with_Z(S0, r, lv, spec.maturity, steps, Z)

    price_gbm, diag_gbm = phoenix_price_vectorized(t_gbm, S_gbm, S0, r, spec)
    price_lv,  diag_lv  = phoenix_price_vectorized(t_lv,  S_lv,  S0, r, spec)

    out = {
        "T_grid": T_grid, "K_grid": K_grid,
        "sigma_imp": sigma_imp, "sigma_loc": sigma_loc,
        "sigma_const": sigma_const,
        "t_gbm": t_gbm, "S_gbm": S_gbm, "diag_gbm": diag_gbm, "price_gbm": price_gbm,
        "t_lv": t_lv, "S_lv": S_lv, "diag_lv": diag_lv, "price_lv": price_lv,
    }
    return out


In [9]:
# Default run
S0 = 100.0
r = 0.03
spec = PhoenixAutocallSpec(
    notional=100.0, maturity=1.0, obs_per_year=12,
    coupon=0.012, B_coupon=0.70, B_call=1.00, B_prot=0.60,
    memory=True, include_coupon_at_call=True
)

bundle = run_pricing_bundle(
    S0=S0, r=r, spec=spec,
    steps=252, n_paths=4000,
    vol_level=0.22, skew=-0.45, curvature=0.70,
    antithetic=True
)

price_gbm = bundle["price_gbm"]
price_lv  = bundle["price_lv"]
sigma_const = bundle["sigma_const"]

summary = pd.DataFrame({
    "Model": ["GBM (const vol)", "Local Vol (Dupire)"],
    "Price": [price_gbm, price_lv],
    "Smile diff (LV-GBM)": [0.0, price_lv - price_gbm],
    "Baseline sigma_const": [sigma_const, sigma_const],
})
summary


,Model,Price,Smile diff (LV-GBM),Baseline sigma_const
0,GBM (const vol),98.959868,0.000000,0.427261
1,Local Vol (Dupire),101.376822,2.416955,0.427261


In [10]:
# Fan charts (visual story)
fig_fan_gbm = fan_chart(bundle["t_gbm"], bundle["S_gbm"], "GBM paths fan chart + barriers", S0, spec)
fig_fan_lv  = fan_chart(bundle["t_lv"],  bundle["S_lv"],  "Local Vol paths fan chart + barriers", S0, spec)
fig_fan_gbm.show(); fig_fan_lv.show()
fig_fan_gbm.write_html(ASSETS_DIR / "p11_ultimate_fan_gbm.html")
fig_fan_lv.write_html(ASSETS_DIR / "p11_ultimate_fan_localvol.html")


In [11]:
# Call probability by date
g_gbm = call_prob_by_date(bundle["diag_gbm"], spec)
g_lv  = call_prob_by_date(bundle["diag_lv"],  spec)

fig_call = go.Figure()
fig_call.add_trace(go.Scatter(x=g_gbm["t"], y=g_gbm["p_call_by_t"], mode="lines+markers", name="GBM"))
fig_call.add_trace(go.Scatter(x=g_lv["t"],  y=g_lv["p_call_by_t"],  mode="lines+markers", name="LocalVol"))
fig_call.update_layout(template="plotly_dark", title="Cumulative call probability by observation date",
                       xaxis_title="t (years)", yaxis_title="P(call by t)")
fig_call.show()
fig_call.write_html(ASSETS_DIR / "p11_ultimate_call_probability.html")


In [12]:
# PV distribution + model-diff distribution
df_pv = pd.DataFrame({
    "GBM": bundle["diag_gbm"]["pv"].values,
    "LocalVol": bundle["diag_lv"]["pv"].values
})
df_melt = df_pv.melt(var_name="Model", value_name="PV")

fig_pv = px.histogram(df_melt, x="PV", color="Model", nbins=80, barmode="overlay",
                      title="PV distribution across Monte Carlo paths")
fig_pv.update_layout(template="plotly_dark")
fig_pv.show()
fig_pv.write_html(ASSETS_DIR / "p11_ultimate_pv_distribution.html")

diff = df_pv["LocalVol"] - df_pv["GBM"]
fig_diff = px.histogram(diff, nbins=80, title="Pathwise model difference: PV(LocalVol) − PV(GBM)")
fig_diff.update_layout(template="plotly_dark", xaxis_title="PV difference")
fig_diff.show()
fig_diff.write_html(ASSETS_DIR / "p11_ultimate_pv_difference_distribution.html")


---  
## 6) Sensitivity heatmap (coupon × autocall barrier)

This is a very strong “structured products” visual:
- coupon \(c\) (higher = more value, but also changes call behavior)
- autocall barrier \(B_{\text{call}}\) (higher = harder to call)

We compute a small grid **fast** (reuse same surface + same random numbers).


In [13]:
def sensitivity_heatmap_coupon_vs_bcall(bundle_ref: dict, S0: float, r: float, spec: PhoenixAutocallSpec,
                                            coupon_grid: np.ndarray, bcall_grid: np.ndarray):
    # Reuse the same Z and the same local vol object to isolate product parameter effect.
    steps = bundle_ref["S_gbm"].shape[0] - 1
    n_paths = bundle_ref["S_gbm"].shape[1]
    # Rebuild Z deterministically (common random numbers)
    Z = np.random.default_rng(SEED).standard_normal((steps, n_paths//2))  # because antithetic doubled it
    Z = np.concatenate([Z, -Z], axis=1)

    # reuse surfaces
    T_grid = bundle_ref["T_grid"]; K_grid = bundle_ref["K_grid"]
    sigma_imp = bundle_ref["sigma_imp"]; sigma_loc = bundle_ref["sigma_loc"]
    lv = LocalVolInterpolator(T_grid, K_grid, sigma_loc)
    sigma_const = bundle_ref["sigma_const"]

    # simulate once
    t_gbm, S_gbm = simulate_gbm_with_Z(S0, r, sigma_const, spec.maturity, steps, Z)
    t_lv,  S_lv  = simulate_localvol_with_Z(S0, r, lv, spec.maturity, steps, Z)

    rows = []
    for cpn in coupon_grid:
        for bc in bcall_grid:
            spec2 = PhoenixAutocallSpec(**{**spec.__dict__, "coupon": float(cpn), "B_call": float(bc)})
            p_gbm, _ = phoenix_price_vectorized(t_gbm, S_gbm, S0, r, spec2)
            p_lv,  _ = phoenix_price_vectorized(t_lv,  S_lv,  S0, r, spec2)
            rows.append({"coupon": cpn, "B_call": bc, "price_gbm": p_gbm, "price_lv": p_lv, "diff": p_lv - p_gbm})

    df = pd.DataFrame(rows)
    return df


coupon_grid = np.linspace(0.006, 0.020, 8)
bcall_grid  = np.linspace(0.95, 1.05, 9)

df_sens = sensitivity_heatmap_coupon_vs_bcall(bundle, S0, r, spec, coupon_grid, bcall_grid)

fig_hm = px.density_heatmap(
    df_sens, x="B_call", y="coupon", z="price_lv", histfunc="avg",
    title="Sensitivity heatmap (LocalVol price) — coupon vs autocall barrier"
)
fig_hm.update_layout(template="plotly_dark", xaxis_title="Autocall barrier B_call", yaxis_title="Coupon c")
fig_hm.show()
fig_hm.write_html(ASSETS_DIR / "p11_ultimate_sensitivity_heatmap.html")

fig_hm2 = px.density_heatmap(
    df_sens, x="B_call", y="coupon", z="diff", histfunc="avg",
    title="Smile impact heatmap: (LocalVol − GBM) price diff"
)
fig_hm2.update_layout(template="plotly_dark", xaxis_title="Autocall barrier B_call", yaxis_title="Coupon c")
fig_hm2.show()
fig_hm2.write_html(ASSETS_DIR / "p11_ultimate_smile_diff_heatmap.html")


---  
## 7) Convergence chart (Monte Carlo stability)

Show how the estimate stabilizes as you increase the number of paths.
This is a strong “quant maturity” signal.


In [14]:
def mc_convergence(S0: float, r: float, spec: PhoenixAutocallSpec, steps: int,
                   vol_level: float, skew: float, curvature: float,
                   path_list=(500, 1000, 2000, 4000, 8000)):
    rows = []
    for n in path_list:
        bundle_n = run_pricing_bundle(S0, r, spec, steps=steps, n_paths=int(n),
                                      vol_level=vol_level, skew=skew, curvature=curvature,
                                      antithetic=True)
        rows.append({"n_paths": int(n)*2, "price_gbm": bundle_n["price_gbm"], "price_lv": bundle_n["price_lv"]})
    return pd.DataFrame(rows)

df_conv = mc_convergence(S0, r, spec, steps=252, vol_level=0.22, skew=-0.45, curvature=0.70,
                         path_list=(400, 800, 1600, 3200, 6400))

fig_conv = go.Figure()
fig_conv.add_trace(go.Scatter(x=df_conv["n_paths"], y=df_conv["price_gbm"], mode="lines+markers", name="GBM"))
fig_conv.add_trace(go.Scatter(x=df_conv["n_paths"], y=df_conv["price_lv"],  mode="lines+markers", name="LocalVol"))
fig_conv.update_layout(template="plotly_dark", title="Monte Carlo convergence (antithetic paths counted)",
                       xaxis_title="Number of paths", yaxis_title="Price estimate")
fig_conv.update_xaxes(type="log")
fig_conv.show()
fig_conv.write_html(ASSETS_DIR / "p11_ultimate_convergence.html")

df_conv


,n_paths,price_gbm,price_lv
0,800,98.478644,101.119935
1,1600,99.366484,101.313899
2,3200,98.976215,101.173935
3,6400,98.814870,101.211122
4,12800,98.872746,101.226909


---  
## 8) Ultimate interactive dashboard (widgets)

This dashboard is the “wow” part:
- tweak smile parameters (vol level / skew / curvature)
- tweak product parameters (coupon & barriers)
- click **Run pricing** → updates:
  - vol sheets (IV + LocalVol)
  - summary table
  - call probability
  - PV distribution + model-diff

If widgets are unavailable, you can skip this section.


In [15]:
def dashboard():
    if not HAS_WIDGETS:
        print("ipywidgets not available in this environment.")
        return

    # Controls
    w_vol = widgets.FloatSlider(value=0.22, min=0.10, max=0.45, step=0.01, description="Vol level")
    w_skew = widgets.FloatSlider(value=-0.45, min=-0.95, max=0.95, step=0.05, description="Skew ρ")
    w_curv = widgets.FloatSlider(value=0.70, min=0.10, max=1.50, step=0.05, description="Curvature")

    w_coupon = widgets.FloatSlider(value=0.012, min=0.002, max=0.03, step=0.001, description="Coupon c")
    w_Bcpn = widgets.FloatSlider(value=0.70, min=0.40, max=1.00, step=0.02, description="B_coupon")
    w_Bcall = widgets.FloatSlider(value=1.00, min=0.85, max=1.15, step=0.01, description="B_call")
    w_Bprot = widgets.FloatSlider(value=0.60, min=0.30, max=1.00, step=0.02, description="B_prot")

    w_paths = widgets.IntSlider(value=3000, min=500, max=10000, step=500, description="n_paths")
    w_steps = widgets.IntSlider(value=252, min=60, max=252, step=6, description="steps")

    w_ant = widgets.Checkbox(value=True, description="Antithetic")

    run_btn = widgets.Button(description="Run pricing", button_style="success")
    out = widgets.Output()

    ui = widgets.VBox([
        widgets.HTML("<b>Smile parameters</b>"),
        widgets.HBox([w_vol, w_skew, w_curv]),
        widgets.HTML("<b>Product parameters</b>"),
        widgets.HBox([w_coupon, w_Bcpn, w_Bcall, w_Bprot]),
        widgets.HTML("<b>Simulation</b>"),
        widgets.HBox([w_paths, w_steps, w_ant, run_btn]),
        out
    ])

    def _run(_):
        with out:
            out.clear_output()
            spec2 = PhoenixAutocallSpec(
                notional=100.0, maturity=1.0, obs_per_year=12,
                coupon=float(w_coupon.value),
                B_coupon=float(w_Bcpn.value),
                B_call=float(w_Bcall.value),
                B_prot=float(w_Bprot.value),
                memory=True,
                include_coupon_at_call=True
            )
            bundle2 = run_pricing_bundle(
                S0=100.0, r=0.03, spec=spec2,
                steps=int(w_steps.value),
                n_paths=int(w_paths.value),
                vol_level=float(w_vol.value),
                skew=float(w_skew.value),
                curvature=float(w_curv.value),
                antithetic=bool(w_ant.value)
            )

            # Surfaces
            fig_iv = plot_vol_surface_3d(bundle2["T_grid"], bundle2["K_grid"], bundle2["sigma_imp"],
                                         "Implied Vol Surface (SVI)", "σ_imp")
            fig_lv = plot_vol_surface_3d(bundle2["T_grid"], bundle2["K_grid"], bundle2["sigma_loc"],
                                         "Local Vol Surface (Dupire)", "σ_loc")
            fig_iv.show(); fig_lv.show()

            # Summary
            summary = pd.DataFrame({
                "Model": ["GBM (const vol)", "Local Vol (Dupire)"],
                "Price": [bundle2["price_gbm"], bundle2["price_lv"]],
                "Smile diff (LV-GBM)": [0.0, bundle2["price_lv"] - bundle2["price_gbm"]],
                "sigma_const": [bundle2["sigma_const"], bundle2["sigma_const"]],
            })
            display(summary)

            # Call prob
            g1 = call_prob_by_date(bundle2["diag_gbm"], spec2)
            g2 = call_prob_by_date(bundle2["diag_lv"], spec2)
            fig_call = go.Figure()
            fig_call.add_trace(go.Scatter(x=g1["t"], y=g1["p_call_by_t"], mode="lines+markers", name="GBM"))
            fig_call.add_trace(go.Scatter(x=g2["t"], y=g2["p_call_by_t"], mode="lines+markers", name="LocalVol"))
            fig_call.update_layout(template="plotly_dark", title="Cumulative call probability by observation date",
                                   xaxis_title="t (years)", yaxis_title="P(call by t)")
            fig_call.show()

            # PV + diff
            df_pv = pd.DataFrame({
                "GBM": bundle2["diag_gbm"]["pv"].values,
                "LocalVol": bundle2["diag_lv"]["pv"].values
            })
            fig_pv = px.histogram(df_pv.melt(var_name="Model", value_name="PV"),
                                  x="PV", color="Model", nbins=70, barmode="overlay",
                                  title="PV distribution")
            fig_pv.update_layout(template="plotly_dark")
            fig_pv.show()

            diff = df_pv["LocalVol"] - df_pv["GBM"]
            fig_diff = px.histogram(diff, nbins=70, title="PV(LocalVol) − PV(GBM)")
            fig_diff.update_layout(template="plotly_dark")
            fig_diff.show()

    run_btn.on_click(_run)
    display(ui)

dashboard()


---  
## 9) What to export for your repo

Inside `/assets/` you now have interactive HTML charts you can link in your README:
- `p11_ultimate_iv_surface_3d.html`  
- `p11_ultimate_localvol_surface_3d.html`  
- `p11_ultimate_smile_animation.html`  
- `p11_ultimate_fan_gbm.html`, `p11_ultimate_fan_localvol.html`  
- `p11_ultimate_call_probability.html`  
- `p11_ultimate_pv_distribution.html`, `p11_ultimate_pv_difference_distribution.html`  
- `p11_ultimate_sensitivity_heatmap.html`, `p11_ultimate_smile_diff_heatmap.html`  
- `p11_ultimate_convergence.html`  

That’s a very clean “quant repo” presentation.
